In [1]:
# Project Description: Plotting GDP Data on a World Map - Part 1

In [2]:
import csv
import math
import pygal

In [3]:
def reconcile_countries_by_name(plot_countries, gdp_countries):
    """
    Inputs:
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country names used in GDP data

    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country names from
      gdp_countries The set contains the country codes from
      plot_countries that were not found in gdp_countries.
    """
    country_code_to_name = {}
    not_found_codes = set()
    
    for code, name in plot_countries.items():
        if name in gdp_countries:
            country_code_to_name[code] = name
        else:
            not_found_codes.add(code)
    
    return country_code_to_name, not_found_codes

In [4]:
def build_map_dict_by_name(gdpinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for

    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    gdp_map = {}
    not_found_countries = set()
    no_gdp_data_countries = set()

    with open(gdpinfo['gdpfile'], 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        gdp_countries = {row[gdpinfo['country_name']]: row for row in reader}
    
    plot_to_gdp_country, not_found_countries = reconcile_countries_by_name(plot_countries, gdp_countries)
    
    for code, country_name in plot_to_gdp_country.items():
        gdp_value = gdp_countries[country_name].get(year, '')
        if gdp_value:
            try:
                gdp_map[code] = math.log10(float(gdp_value))
            except ValueError:
                no_gdp_data_countries.add(code)
        else:
            no_gdp_data_countries.add(code)
    
    return gdp_map, not_found_countries, no_gdp_data_countries

In [5]:
# Example GDP info dictionary
# gdpinfo = {
#     'gdpfile': 'path_to_your_gdp_file.csv',
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code',
#     'country_names': {name: {} for name in gdp_country_names}
# }

# # Example plot_countries dictionary
# plot_countries = {
#     'us': 'United States',
#     'ca': 'Canada',
#     'mx': 'Mexico',
#     'fr': 'France',
#     'de': 'Germany',
#     'cn': 'China'
# }

# # Example year
# year = '2010'

# # Run the function
# result = build_map_dict_by_name(gdpinfo, plot_countries, year)
# print(result)

In [6]:
def render_world_map(gdpinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for
      map_file       - Name of output file to create

    Output:
      Returns None.

    Action:
      Creates a world map plot of the GDP data for the given year and
      writes it to a file named by map_file.
    """
    # Get the GDP data
    gdp_map, not_found_countries, no_gdp_data_countries = build_map_dict_by_name(gdpinfo, plot_countries, year)

    # Create a world map
    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = f'World GDP in {year}'

    # Add data to the map
    worldmap_chart.add('GDP (log scale)', gdp_map)
    worldmap_chart.add('Missing from World Bank Data', list(not_found_countries))
    worldmap_chart.add('No GDP Data for Year', list(no_gdp_data_countries))

    # Render the map to a file
    worldmap_chart.render_to_file(map_file)

In [7]:
# Example GDP info dictionary
# gdpinfo = {
#     'gdpfile': 'path_to_your_gdp_file.csv',
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code'
# }

# # Example plot_countries dictionary
# plot_countries = {
#     'us': 'United States',
#     'ca': 'Canada',
#     'mx': 'Mexico',
#     'fr': 'France',
#     'de': 'Germany',
#     'cn': 'China'
# }

# # Example year
# year = '2010'

# # Example map file
# map_file = 'world_gdp_2010.svg'

# # Run the function to render the map
# render_world_map(gdpinfo, plot_countries, year, map_file)

In [8]:
def test_render_world_map():
    """
    Test the project code for several years.
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    # Get pygal country code map
    pygal_countries = pygal.maps.world.COUNTRIES

    # 1960
    render_world_map(gdpinfo, pygal_countries, "1960", "isp_gdp_world_name_1960.svg")

    # 1980
    render_world_map(gdpinfo, pygal_countries, "1980", "isp_gdp_world_name_1980.svg")

    # 2000
    render_world_map(gdpinfo, pygal_countries, "2000", "isp_gdp_world_name_2000.svg")

    # 2010
    render_world_map(gdpinfo, pygal_countries, "2010", "isp_gdp_world_name_2010.svg")

In [9]:
# Make sure the following call to test_render_world_map is commented
# out when submitting to OwlTest/CourseraTest.

test_render_world_map()

In [10]:
# Project Description: Plotting GDP Data on a World Map - Part 2

In [11]:
def build_country_code_converter(codeinfo):
    """
    Inputs:
      codeinfo      - A country code information dictionary

    Output:
      A dictionary whose keys are plot country codes and values
      are world bank country codes, where the code fields in the
      code file are specified in codeinfo.
    """
    converter = {}
    
    with open(codeinfo['codefile'], 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=codeinfo['separator'], quotechar=codeinfo['quote'])
        for row in reader:
            plot_code = row[codeinfo['plot_codes']].strip()
            data_code = row[codeinfo['data_codes']].strip()
            converter[plot_code] = data_code
    
    return converter

In [12]:
# Example codeinfo dictionary
# codeinfo = {
#     'codefile': 'path_to_your_country_codes_file.csv',
#     'separator': ',',
#     'quote': '"',
#     'plot_codes': 'Plot Country Code',
#     'data_codes': 'Data Country Code'
# }

# # Run the function to build the country code converter
# converter = build_country_code_converter(codeinfo)
# print(converter)

In [13]:
def reconcile_countries_by_code(codeinfo, plot_countries, gdp_countries):
    """
    Inputs:
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country codes used in GDP data

    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country codes from
      gdp_countries.  The set contains the country codes from
      plot_countries that did not have a country with a corresponding
      code in gdp_countries.

      Note that all codes should be compared in a case-insensitive
      way.  However, the returned dictionary and set should include
      the codes with the exact same case as they have in
      plot_countries and gdp_countries.
    """
    converter = build_country_code_converter(codeinfo)
    plot_to_gdp = {}
    not_found = set()
    
    gdp_countries_lower = {key.lower(): key for key in gdp_countries.keys()}
    converter_lower = {key.lower(): value.lower() for key, value in converter.items()}
    
    for plot_code, country_name in plot_countries.items():
        plot_code_lower = plot_code.lower()
        if plot_code_lower in converter_lower:
            data_code_lower = converter_lower[plot_code_lower]
            if data_code_lower in gdp_countries_lower:
                plot_to_gdp[plot_code] = gdp_countries_lower[data_code_lower]
            else:
                not_found.add(plot_code)
        else:
            not_found.add(plot_code)
    
    return plot_to_gdp, not_found

In [14]:
# Example usage:

# Example codeinfo dictionary
# codeinfo = {
#     'codefile': 'path_to_your_country_codes_file.csv',
#     'separator': ',',
#     'quote': '"',
#     'plot_codes': 'Plot Country Code',
#     'data_codes': 'Data Country Code'
# }

# # Example plot_countries dictionary
# plot_countries = {
#     'US': 'United States',
#     'CA': 'Canada',
#     'MX': 'Mexico',
#     'FR': 'France',
#     'DE': 'Germany',
#     'CN': 'China'
# }

# # Example gdp_countries dictionary
# gdp_countries = {
#     'USA': {},
#     'CAN': {},
#     'MEX': {},
#     'FRA': {},
#     'DEU': {},
#     'CHN': {}
# }

# # Run the function to reconcile countries by code
# result = reconcile_countries_by_code(codeinfo, plot_countries, gdp_countries)
# print(result)

In [15]:
def build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary mapping plot library country codes to country names
      year           - String year for which to create GDP mapping

    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    gdp_map = {}
    not_found_countries = set()
    no_gdp_data_countries = set()
    
    # Load the GDP data
    with open(gdpinfo['gdpfile'], 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=gdpinfo['separator'], quotechar=gdpinfo['quote'])
        gdp_data = {row[gdpinfo['country_code']].strip(): row for row in reader}
    
    # Reconcile countries
    plot_to_gdp, not_found_countries = reconcile_countries_by_code(codeinfo, plot_countries, gdp_data)
    
    # Process the GDP data
    for plot_code, gdp_code in plot_to_gdp.items():
        gdp_value = gdp_data[gdp_code].get(year, '')
        if gdp_value:
            try:
                gdp_map[plot_code] = math.log10(float(gdp_value))
            except ValueError:
                no_gdp_data_countries.add(plot_code)
        else:
            no_gdp_data_countries.add(plot_code)
    
    return gdp_map, not_found_countries, no_gdp_data_countries

In [16]:
# Example gdpinfo dictionary
# gdpinfo = {
#     'gdpfile': 'path_to_your_gdp_file.csv',
#     'separator': ',',
#     'quote': '"',
#     'min_year': 2010,
#     'max_year': 2017,
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code'
# }

# # Example codeinfo dictionary
# codeinfo = {
#     'codefile': 'path_to_your_country_codes_file.csv',
#     'separator': ',',
#     'quote': '"',
#     'plot_codes': 'Plot Country Code',
#     'data_codes': 'Data Country Code'
# }

# # Example plot_countries dictionary
# plot_countries = {
#     'US': 'United States',
#     'CA': 'Canada',
#     'MX': 'Mexico',
#     'FR': 'France',
#     'DE': 'Germany',
#     'CN': 'China'
# }

# # Example year
# year = '2010'

# # Run the function to build the map dictionary by code
# result = build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year)
# print(result)

In [17]:
def render_world_map(gdpinfo, codeinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary mapping plot library country codes to country names
      year           - String year of data
      map_file       - String that is the output map file name

    Output:
      Returns None.

    Action:
      Creates a world map plot of the GDP data in gdp_mapping and outputs
      it to a file named by svg_filename.
    """
    # Get the GDP data
    gdp_map, not_found_countries, no_gdp_data_countries = build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year)

    # Create a world map
    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = f'World GDP in {year}'

    # Add data to the map
    worldmap_chart.add('GDP (log scale)', gdp_map)
    worldmap_chart.add('Missing from World Bank Data', list(not_found_countries))
    worldmap_chart.add('No GDP Data for Year', list(no_gdp_data_countries))

    # Render the map to a file
    worldmap_chart.render_to_file(map_file)

In [18]:
# Example usage:

# Example gdpinfo dictionary
# gdpinfo = {
#     'gdpfile': 'path_to_your_gdp_file.csv',
#     'separator': ',',
#     'quote': '"',
#     'min_year': 2010,
#     'max_year': 2017,
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code'
# }

# # Example codeinfo dictionary
# codeinfo = {
#     'codefile': 'path_to_your_country_codes_file.csv',
#     'separator': ',',
#     'quote': '"',
#     'plot_codes': 'Plot Country Code',
#     'data_codes': 'Data Country Code'
# }

# # Example plot_countries dictionary
# plot_countries = {
#     'US': 'United States',
#     'CA': 'Canada',
#     'MX': 'Mexico',
#     'FR': 'France',
#     'DE': 'Germany',
#     'CN': 'China'
# }

# # Example year
# year = '2010'

# # Example map file
# map_file = 'world_gdp_2010.svg'

# # Run the function to render the map
# render_world_map(gdpinfo, codeinfo, plot_countries, year, map_file)

In [19]:
def test_render_world_map():
    """
    Test the project code for several years
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    codeinfo = {
        "codefile": "isp_country_codes.csv",
        "separator": ",",
        "quote": '"',
        "plot_codes": "ISO3166-1-Alpha-2",
        "data_codes": "ISO3166-1-Alpha-3"
    }

    # Get pygal country code map
    pygal_countries = pygal.maps.world.COUNTRIES

    # 1960
    render_world_map(gdpinfo, codeinfo, pygal_countries, "1960", "isp_gdp_world_code_1960.svg")

    # 1980
    render_world_map(gdpinfo, codeinfo, pygal_countries, "1980", "isp_gdp_world_code_1980.svg")

    # 2000
    render_world_map(gdpinfo, codeinfo, pygal_countries, "2000", "isp_gdp_world_code_2000.svg")

    # 2010
    render_world_map(gdpinfo, codeinfo, pygal_countries, "2010", "isp_gdp_world_code_2010.svg")

In [20]:
# Make sure the following call to test_render_world_map is commented
# out when submitting to OwlTest/CourseraTest.

test_render_world_map()